In [4]:
#####code was built/leveraged the following resources: Coinbase API Docs (https://docs.pro.coinbase.com/), Coinbasepro-python Github(https://github.com/danpaquin/coinbasepro-python)

#####Purpose: the following code implements a 5-minute time-frame/5-minute window Simple Moving Aveage in order to produce buy/sell signals on BTC-USD

###Import modules and libraries
import json, hmac, hashlib, time, requests, base64
import datetime, timedelta
import pandas as pd 
import numpy as np 
import sched, time
from requests.auth import AuthBase

### Create custom authentication for Exchange
class CoinbaseExchangeAuth(AuthBase):
    def __init__(self, api_key, secret_key, passphrase):
        self.api_key = api_key
        self.secret_key = secret_key
        self.passphrase = passphrase

    def __call__(self, request):
        timestamp = str(time.time())
        message = timestamp + request.method + request.path_url + (request.body or b'').decode()      
        message = bytes(message, 'utf-8')
        secret = self.secret_key
        signature = base64.b64encode(hmac.new(secret, message, digestmod=hashlib.sha256).digest()).rstrip(b'\n')

        request.headers.update({
            'CB-ACCESS-SIGN': signature,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'
        })
        return request

   
##### Declare inputs to variables 
api_url = 'https://api.gdax.com/'

API_KEY = bytes('key_goes_here', 'utf8')
API_SECRET = base64.b64decode(b'{secret_goes_here}')
API_PASS = bytes('pass_goes_here', 'utf8')

auth = CoinbaseExchangeAuth(API_KEY, API_SECRET,  API_PASS)

price_to_get = 'BTC-USD'

start_date_days_miunus = datetime.timedelta(days=1)
start_date=datetime.datetime.now()-start_date_days_miunus
iso_start_date=start_date.isoformat()
end_date=datetime.datetime.now().isoformat()
end_date=datetime.datetime.now().isoformat()

historical_price = {
    'start': iso_start_date,
    'end': end_date,
    'granularity': 300 #900=15mins
}


#####Get historical prices/run Algo
r = requests.get(api_url + 'products/'+price_to_get+'/candles', params=historical_price)
tag = 'getting historical price'

if (tag == 'place order'):
    return_data = r.json()
    trade_id= (return_data["id"])
else:
    trade_id='not placing an order'

json_data = json.loads(r.text)

###begin Algo logic/building of historical results
if (tag == 'getting historical price'):
    df = 1
    del df
    df = pd.DataFrame(json_data,columns = [ 'time', 'low', 'high','open', 'close', 'volume']) 
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.sort_values(by='time', ascending=True, inplace=True)
    df['SMA'] = df['close'].rolling(5).mean()
    df['SMA_prev'] = df['SMA'].shift()
    df['action'] = np.where((df['open']>df['SMA_prev']), 'buy'
                            , np.where((df['open']<df['SMA_prev']), 'sell', None))
    df['action_prev'] = df['action'].shift()
    df['action_two'] = np.where((df['action']=='buy') & (df['action_prev']!='buy') ,'buy','None')
    df['buy_count'] = df['action_two'].eq('buy').cumsum()
    df['action_three'] = np.where((df['action']=='buy') & (df['action_two']=='buy') & (df['action'].shift()!='buy'), 'buy'
                            , np.where((df['action']=='sell') & (df['action_two'] == 'None') & (df['action'].shift()!='sell') & (df['buy_count']>=1), 'sell',None))
    df['action_price'] = np.where((df['action_three']=='buy') | (df['action_three']=='sell'), df['open'],None) 

###print results
    print(df, end='\r')
else:
    df = None 
    print(json.dumps(json_data,sort_keys=True,indent=4))


                   time      low     high     open    close     volume  \
287 2020-07-19 18:00:00  9152.50  9161.27  9154.35  9159.67   7.870735   
286 2020-07-19 18:05:00  9152.91  9159.67  9159.67  9154.23   4.983009   
285 2020-07-19 18:10:00  9153.12  9155.37  9154.80  9153.18   7.578602   
284 2020-07-19 18:15:00  9153.12  9154.91  9153.19  9154.90   7.933007   
283 2020-07-19 18:20:00  9153.82  9159.63  9154.23  9158.89   6.818309   
..                  ...      ...      ...      ...      ...        ...   
4   2020-07-20 17:35:00  9193.99  9209.66  9193.99  9205.21  64.324412   
3   2020-07-20 17:40:00  9194.31  9205.00  9205.00  9195.96  29.125888   
2   2020-07-20 17:45:00  9195.95  9199.99  9195.96  9199.99  23.144476   
1   2020-07-20 17:50:00  9196.57  9202.95  9199.98  9196.58  31.113553   
0   2020-07-20 17:55:00  9193.93  9197.63  9196.58  9196.04  21.356841   

          SMA  SMA_prev action action_prev action_two  buy_count action_three  \
287       NaN       NaN   None